## 전처리
ㅇ불필요한 단어 제거 및 문장 단위로 구분하여 저장

In [ ]:
import pandas as pd
df=pd.read_csv('카테고리_상품.csv', header=0, encoding='utf-8', index_col=0)
df

In [ ]:
#쓸데없는 메모리 축소 함수
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    import numpy as np
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    return df

In [ ]:
df = reduce_mem_usage(df)
df["review"].iloc[0]

In [ ]:
df.info()

In [ ]:
df=df.drop_duplicates()

In [ ]:
df=df.drop(df[df['review'].isnull()].index)
df=df.reset_index(drop=True)
df.info()

In [ ]:
from konlpy.tag import Okt

okt = Okt()
okt.morphs(df['review'].loc[1])

In [ ]:
len(df)

In [ ]:
df['doc'] = df['review'].str.replace("[^ㄱ-하-ㅣ가-힣]", " ")
df

In [ ]:
import numpy as np
df['doc'].replace("", np.nan, inplace=True)

In [ ]:
(df['review'].loc[1])

In [ ]:
from konlpy.tag import Okt

#-*- coding: utf-8 -*-
f = open("korean_stopwords.txt", 'r', encoding='UTF8')
read = f.read()
stop_words = read.split()

name ="상품명"
okt = Okt()
st= okt.morphs(name)

for i in range(len(stop_words)):
    st.append(stop_words[i])
st

In [ ]:
def clean(dfdf):
    okt = Okt()
    dfdf=reduce_mem_usage(dfdf)
    dfdf['word']=0
    for i in range(len(dfdf)):
        lists=[] 
        t_words = okt.nouns(dfdf['doc'].iloc[i])
        for w in t_words:
            if w not in st:
                lists.append(w)
        dfdf['word'].loc[i]=lists
    return dfdf

In [ ]:
df_clean=clean(df)
df_clean

## 키워드분석
o 빈도 수가 높은 키워드(Counter)

In [ ]:
word=[]
for i in range(len(df_clean)):
    for j in range(len(df_clean['word'].iloc[i])):
        word.append(df_clean['word'].iloc[i][j])
len(word)

In [ ]:
from collections import Counter
counter = Counter(word)
most_common = counter.most_common(15)
most_common

In [ ]:
for i in range(5):
    print(most_common[i][0])

## 키워드 분석
o 긍정부정 별 키워드 (TF-IDF)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import re

def text_cleaning(text):
    words = re.compile('[^ ㄱ-ㅣ 가-힣]')
    result = words.sub('', text)
    okt = Okt() 
    nouns = okt.nouns(result)
    nouns = [x for x in nouns if len(x) > 1] 
    nouns = [x for x in nouns if x not in stop_words]
    return nouns

vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))
bow_vect = vect.fit_transform(df['review'].tolist())

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect_model = TfidfVectorizer()
vect_model.fit(word)

In [ ]:
tf_idf_vect.shape

In [ ]:
vect.vocabulary_

In [ ]:
import numpy as np
conditionlist = [(df_clean['score'] >= 4), (df_clean['score'] == 3), (df_clean['score'] <= 2)]
choicelist = [1, 0, -1]
df_clean['rate'] = np.select(conditionlist, choicelist, default='Not Specified')
df_clean

In [ ]:
df_test = df_clean.loc[df_clean['rate']== '0']
df_train = df_clean[(df_clean['rate'] == '1') | (df_clean['rate'] == '-1')]

In [ ]:
df["rate"].value_counts()

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

bow_vect = vect.fit_transform(df_train['doc'].tolist())
tfidf_vectorizer = TfidfTransformer()
tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)

In [ ]:
from sklearn.model_selection import train_test_split

x = tf_idf_vect
y = df_train['rate']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

In [ ]:
print("x_train", x_train.shape)
print("x_test", x_test.shape)
print("y_train", y_train.shape)
print("y_test", y_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr = LogisticRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_test)

In [ ]:
print('정확도: %.2f' % accuracy_score(y_test, y_pred))

In [ ]:
lr.coef_

In [ ]:
coef_pos_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = True)
coef_neg_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = False)
coef_pos_index

In [ ]:
invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}
invert_index_vectorizer

In [ ]:
for coef in coef_pos_index[:10]:
    print(invert_index_vectorizer[coef[1]], coef[0])

In [ ]:
for coef in coef_neg_index[:10]:
    print(invert_index_vectorizer[coef[1]], coef[0])

In [ ]:
pos=[]
for coef in coef_pos_index[:10]:
    pos.append(invert_index_vectorizer[coef[1]])

neg=[]
for coef in coef_neg_index[:10]:
    neg.append(invert_index_vectorizer[coef[1]])
    
final = pd.DataFrame({"긍정" : pos, "부정" : neg})
final